# Load Files

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Notebooks
import nbimporter
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Functions Notebook
from Utils import build_GAN, batch_build_GAN, corr_matrix
from Functions import holisticsTrainTest
from Functions import comfPMV, ppv, selectModelParameters, trainTest_tunedModel, buildTrainRF, buildTrainKNN
from Functions import buildTrainSVM, buildTrainNB, buildMLP, getClfMetrics, saveModel

# Pandas, matplotlib, pickle, seaborn
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# from collections import Counter

# Sklearn
from sklearn.manifold import TSNE
from sklearn import preprocessing

# Tensorflow
import tensorflow as tf

Importing Jupyter notebook from Utils.ipynb
Importing Jupyter notebook from GAN.ipynb
Importing Jupyter notebook from Functions.ipynb


In [2]:
# Run the pre-processing notebook first to make sure the pickle files exist

df_feature1 = pd.read_pickle("df_feature1.pkl")
# df_feature2 = pd.read_pickle("df_feature2.pkl")
# df_feature3 = pd.read_pickle("df_feature3.pkl")
# df_feature4 = pd.read_pickle("df_feature4.pkl")
# df_feature5 = pd.read_pickle("df_feature5.pkl")


In [3]:
test_size_percentage = 0.2 # for CV within train split
train_test_split = 0.8 # for main train/validation split

user_split = False #split train and test user-based or completely stratfied

use_heuristics_participants = False


In [4]:
df_aux_65 = pd.read_csv("../data/TCS_65_participants_outsideData.csv", delimiter = ",")

list_complete_participants = list(df_aux_65['Participant_No'].unique())

if use_heuristics_participants:
    list_complete_participants.append(10)
    list_complete_participants.append(26)
    list_complete_participants.append(28)
    list_complete_participants.append(36)
    list_complete_participants.append(37)
    list_complete_participants.append(39)
    list_complete_participants.append(47)
    list_complete_participants.append(48)
    list_complete_participants.append(53)

num_complete_participants = len(list_complete_participants)
print("Complete Participants: {}".format(list_complete_participants))
print("Total number of complete participants: {}".format(num_complete_participants))

Complete Participants: [1, 2, 4, 5, 6, 7, 8, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 29, 30, 31, 32, 33, 34, 35, 40, 41, 42, 43, 44, 45, 46, 49, 50, 51, 52, 54, 55, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80]
Total number of complete participants: 65


In [20]:

df_feature1_train, df_feature1_test, df_feature1_train_binary, df_feature1_test_binary = holisticsTrainTest(df_feature1, 
                                                                                                            list_complete_participants,
                                                                                                            user_split = user_split,
                                                                                                            train_test_split = train_test_split)


# df_feature2_train, df_feature2_test, df_feature2_train_binary, df_feature2_test_binary  = holisticsTrainTest(df_feature2, 
#                                                                                                              list_complete_participants,
#                                                                                                             user_split = user_split,
#                                                                                                             train_test_split=train_test_split)

# df_feature3_train, df_feature3_test, df_feature3_train_binary, df_feature3_test_binary = holisticsTrainTest(df_feature3,
#                                                                                                            list_complete_participants,
#                                                                                                            user_split = user_split,
#                                                                                                            train_test_split=train_test_split)

# df_feature4_train, df_feature4_test, df_feature4_train_binary, df_feature4_test_binary = holisticsTrainTest(df_feature4,
#                                                                                                            list_complete_participants,
#                                                                                                            user_split = user_split,
#                                                                                                            train_test_split=train_test_split)

# df_feature5_train, df_feature5_test, df_feature5_train_binary, df_feature5_test_binary = holisticsTrainTest(df_feature5,
#                                                                                                             list_complete_participants,
#                                                                                                            user_split = user_split,
#                                                                                                            train_test_split=train_test_split)

print("Total number of instances: {}".format(df_feature1.shape[0]))
print(df_feature1['Discrete Thermal Comfort_TA'].value_counts())
print("Number of training instances: {}".format(df_feature1_train.shape[0]))
print("Number of testing (validation) instances: {}".format(df_feature1_test.shape[0]))

Total number of instances: 2067
 0    1152
-1     452
-2     217
 1     198
 2      48
Name: Discrete Thermal Comfort_TA, dtype: int64
Number of training instances: 1653
Number of testing (validation) instances: 414


# Preparing data

In [6]:
print(df_feature1_train['Discrete Thermal Comfort_TA'].value_counts())
print(df_feature1_test['Discrete Thermal Comfort_TA'].value_counts())
# df_feature1_train.to_csv('df_feature1_train.csv', index=False)

 0    918
-1    372
-2    173
 1    157
 2     33
Name: Discrete Thermal Comfort_TA, dtype: int64
 0    234
-1     80
-2     44
 1     41
 2     15
Name: Discrete Thermal Comfort_TA, dtype: int64


In [32]:
# needed datapoints per label
needed_0 = 950
needed_minus1 = 353 * 3
needed_minus2 = 190 * 3
needed_1 = 176 * 3
needed_2 = 44 * 3

print(needed_minus1)
print(needed_minus2)
print(needed_1)
print(needed_2)

1059
570
528
132


## Occutherm thermal comfort models

In [33]:
# load best models NB and KNN
acc_nb_1_train, nb_optimal_1 = buildTrainNB(df_feature1_train, test_size_percentage=test_size_percentage)
acc_knn_1_train, knn_optimal_1 = buildTrainKNN(df_feature1_train,test_size_percentage=test_size_percentage)
acc_svm_1_train, svm_optimal_1 = buildTrainSVM(df_feature1_train, test_size_percentage=test_size_percentage)

# using the optimal model. re-train in whole train split and test in unseen test split
acc_holistic_knn_1, _ = trainTest_tunedModel(df_feature1_test, knn_optimal_1)
acc_holistic_nb_1, _ = trainTest_tunedModel(df_feature1_test, nb_optimal_1)
acc_holistic_svm_1, _= trainTest_tunedModel(df_feature1_test, svm_optimal_1)

print(acc_holistic_knn_1)
print(acc_holistic_nb_1)
print(acc_holistic_svm_1)


Features: ['Temperature (Fahrenheit)' 'SkinTemperature' 'ClothingInsulation'
 'Height(cm)' 'Shoulder Circumference(cm)' 'Weight(lbs)' 'Gender'
 'Temperature_outside' 'Humidity_outside']
Expected accuracy (f1 micro) based on Cross-Validation:  0.5266228621517893
GaussianNB(priors=None, var_smoothing=1e-09)
Accuracy (f1 micro) on test set:  0.513595166163142
F1 micro on test set:  0.513595166163142
F1 macro on test set:  0.44454532230737226
Confusion Matrix: 
[[ 21   9   5   0   0]
 [ 19  28  27   0   0]
 [ 19  34 107  21   3]
 [  1   0  18  11   1]
 [  0   0   3   1   3]]
Classification Metrics: 
              precision    recall  f1-score   support

          -2       0.35      0.60      0.44        35
          -1       0.39      0.38      0.39        74
           0       0.67      0.58      0.62       184
           1       0.33      0.35      0.34        31
           2       0.43      0.43      0.43         7

   micro avg       0.51      0.51      0.51       331
   macro avg     

/home/matias/anaconda3/envs/tGAN/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best parameters set found on development set:
{'algorithm': 'brute', 'metric': 'seuclidean', 'n_neighbors': 3, 'weights': 'distance'}
KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='seuclidean',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='distance')
Accuracy (f1 micro) on test set:  0.6858006042296072
F1 micro on test set:  0.6858006042296072
F1 macro on test set:  0.5602459224188084
Confusion Matrix: 
[[ 24   8   3   0   0]
 [ 12  38  24   0   0]
 [  3  26 147   7   1]
 [  0   0  11  16   4]
 [  0   0   1   4   2]]
Classification Metrics: 
              precision    recall  f1-score   support

          -2       0.62      0.69      0.65        35
          -1       0.53      0.51      0.52        74
           0       0.79      0.80      0.79       184
           1       0.59      0.52      0.55        31
           2       0.29      0.29      0.29         7

   micro avg       0.69      0.69      0.69       331
   macro avg       

/home/matias/anaconda3/envs/tGAN/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best parameters set found on development set:
{'C': 1000, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'}
SVC(C=1000, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=100, shrinking=True,
  tol=0.001, verbose=False)
Accuracy (f1 micro) on test set:  0.6827794561933535
F1 micro on test set:  0.6827794561933535
F1 macro on test set:  0.5749621346395539
Confusion Matrix: 
[[ 25  10   0   0   0]
 [ 11  47  15   1   0]
 [  6  35 134   8   1]
 [  1   2   7  18   3]
 [  0   0   1   4   2]]
Classification Metrics: 
              precision    recall  f1-score   support

          -2       0.58      0.71      0.64        35
          -1       0.50      0.64      0.56        74
           0       0.85      0.73      0.79       184
           1       0.58      0.58      0.58        31
           2       0.33      0.29      0.31         7

   micro avg       0.68      0.6

/home/matias/anaconda3/envs/tGAN/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matias/anaconda3/envs/tGAN/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matias/anaconda3/envs/tGAN/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matias/anaconda3/envs/tGAN/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted

# tGAN 

In [9]:
from tgan.model import TGANModel
# continuous_columns, that will contain a list with the indices of continuous columns.
# df_feature1_train.to_csv("../data/df_feature1_train.csv", index=False)
# print(df_feature1_train)
continuous_columns = [0, 1, 2, 3, 4, 5, 7, 8]


W0811 10:37:57.360398 139778422806336 module_wrapper.py:136] From /home/matias/anaconda3/envs/tGAN/lib/python3.7/site-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.train.SessionRunHook is deprecated. Please use tf.estimator.SessionRunHook instead.



In [10]:
"""best architecture based on TGAN architecture search
{
"batch_size": 100, 
"z_dim": 100, 
"num_gen_rnn": 400, 
"num_gen_feature": 100, 
"num_dis_layers": 5, 
"num_dis_hidden": 200, 
"learning_rate": 0.0002, 
"noise": 0.1, 
"max_epoch": 5, 
"steps_per_epoch": 1000, 
"score": 0.5043731778425656
}
"""

'best architecture based on TGAN architecture search\n{\n"batch_size": 100, \n"z_dim": 100, \n"num_gen_rnn": 400, \n"num_gen_feature": 100, \n"num_dis_layers": 5, \n"num_dis_hidden": 200, \n"learning_rate": 0.0002, \n"noise": 0.1, \n"max_epoch": 5, \n"steps_per_epoch": 1000, \n"score": 0.5043731778425656\n}\n'

In [11]:
tgan = TGANModel(
    continuous_columns,
    output='output',
    max_epoch=5,
    steps_per_epoch=1000,
    save_checkpoints=True,
    restore_session=True,
    batch_size=100,
    z_dim=100,
    noise=0.1,
    learning_rate=0.0002,
    num_gen_rnn=400,
    num_gen_feature=100,
    num_dis_layers=5,
    num_dis_hidden=200,
    )

tgan.fit(df_feature1_train) # 

W0811 10:37:57.511141 139778422806336 module_wrapper.py:136] From /home/matias/anaconda3/envs/tGAN/lib/python3.7/site-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



[0811 10:37:57 @input_source.py:222] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...


W0811 10:37:57.543535 139778422806336 module_wrapper.py:136] From /home/matias/anaconda3/envs/tGAN/lib/python3.7/site-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.summary.scalar is deprecated. Please use tf.compat.v1.summary.scalar instead.

W0811 10:37:57.550619 139778422806336 deprecation.py:323] From /home/matias/anaconda3/envs/tGAN/lib/python3.7/site-packages/tgan/model.py:259: LSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0811 10:37:57.572559 139778422806336 deprecation.py:323] From /home/matias/anaconda3/envs/tGAN/lib/python3.7/site-packages/tensorflow_core/python/ops/rnn_cell_impl.py:957: Layer.add_variable (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.ad

[0811 10:37:57 @registry.py:126] gen/LSTM/00/FC input: [100, 400]


W0811 10:37:57.597186 139778422806336 module_wrapper.py:136] From /home/matias/anaconda3/envs/tGAN/lib/python3.7/site-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.layers.Dense is deprecated. Please use tf.compat.v1.layers.Dense instead.

W0811 10:37:57.597885 139778422806336 deprecation.py:323] From /home/matias/anaconda3/envs/tGAN/lib/python3.7/site-packages/tensorpack/models/fc.py:67: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.


...


100%|##########|1000/1000[06:52<00:00, 2.43it/s]

[0811 11:05:48 @base.py:285] Epoch 4 (global_step 4000) finished, time:6 minutes 52 seconds.
[0811 11:05:48 @saver.py:79] Model saved to output/model/model-4000.
[0811 11:05:48 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 1
[0811 11:05:48 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0
[0811 11:05:48 @monitor.py:467] GAN_loss/discrim/loss: 0.67072
[0811 11:05:48 @monitor.py:467] GAN_loss/gen/final-g-loss: 1.1488
[0811 11:05:48 @monitor.py:467] GAN_loss/gen/klloss: 0.23303
[0811 11:05:48 @monitor.py:467] GAN_loss/gen/loss: 0.91579
[0811 11:05:48 @monitor.py:467] QueueInput/queue_size: 50
[0811 11:05:48 @base.py:275] Start Epoch 5 ...



100%|##########|1000/1000[06:52<00:00, 2.43it/s]

[0811 11:12:40 @base.py:285] Epoch 5 (global_step 5000) finished, time:6 minutes 52 seconds.
[0811 11:12:40 @saver.py:79] Model saved to output/model/model-5000.
[0811 11:12:40 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 1
[0811 11:12:40 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0
[0811 11:12:40 @monitor.py:467] GAN_loss/discrim/loss: 0.67062
[0811 11:12:40 @monitor.py:467] GAN_loss/gen/final-g-loss: 1.0985
[0811 11:12:40 @monitor.py:467] GAN_loss/gen/klloss: 0.18
[0811 11:12:40 @monitor.py:467] GAN_loss/gen/loss: 0.91853
[0811 11:12:40 @monitor.py:467] QueueInput/queue_size: 50
[0811 11:12:40 @base.py:289] Training has finished!


[0811 11:12:40 @input_source.py:178] EnqueueThread QueueInput/input_queue Exited.
[0811 11:12:42 @collection.py:146] New collections created in tower : tf.GraphKeys.REGULARIZATION_LOSSES
[0811 11:12:42 @collection.py:165] These collections were modified but restored in : (tf.GraphKeys.SUMMARIES: 0->2)
[0811 11:12:42 @sessinit.py:87] WRN The following variables are in the checkpoint, but not found in the graph: global_step, optimize/beta1_power, optimize/beta2_power
[0811 11:12:43 @sessinit.py:114] Restoring checkpoint from output/model/model-5000 ...


## Generate samples

In [12]:
def sample_needed_label(num_samples, label):
    synth_df = pd.DataFrame(columns=df_feature1.columns.values[1:])
    count = 0
    finish_loop = False
    # generate as many sampels of label 0 as needed
    while True:
        if finish_loop:
            break

        # sample a bunch of points
        curr_df = tgan.sample(1000)
        # iterate through all
        for index, row in curr_df.iterrows():
            if row['Discrete Thermal Comfort_TA'] == label:
                synth_df = synth_df.append(row)
                count += 1
                print(count)
                if count == num_samples:
                    finish_loop = True
                    break
    return synth_df


In [13]:
print("Samples needed: {}".format(needed_0))
synth_0 = sample_needed_label(needed_0, '0')


  0%|          |0/100[00:00<?,?it/s]

Samples needed: 950


  9%|9         |9/100[00:01<00:13, 6.86it/s]


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [34]:
print("Samples needed: {}".format(needed_minus1))
synth_minus1 = sample_needed_label(needed_minus1, '-1')

  0%|          |0/100[00:00<?,?it/s]

Samples needed: 1059


  0%|          |0/100[00:00<?,?it/s]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


  0%|          |0/100[00:00<?,?it/s]

...

1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046


  0%|          |0/100[00:00<?,?it/s]

1047
1048
1049
1050
1051


  9%|9         |9/100[00:01<00:10, 8.91it/s]

1052
1053
1054
1055
1056
1057
1058
1059


In [ ]:
print("Samples needed: {}".format(needed_minus2))
synth_minus2 = sample_needed_label(needed_minus2, '-2')

  0%|          |0/100[00:00<?,?it/s]

Samples needed: 570


  0%|          |0/100[00:00<?,?it/s]

1


  0%|          |0/100[00:00<?,?it/s]

...

  0%|          |0/100[00:00<?,?it/s]

212


  0%|          |0/100[00:00<?,?it/s]

213


  0%|          |0/100[00:00<?,?it/s]

In [ ]:
print("Samples needed: {}".format(needed_1))
synth_1 = sample_needed_label(needed_1, '1')

In [ ]:
print("Samples needed: {}".format(needed_2))
synth_2 = sample_needed_label(needed_2, '2')

In [ ]:
# df_synth = pd.concat([synth_0, synth_minus1, synth_minus2, synth_1, synth_2], ignore_index=True)
df_synth = pd.concat([synth_minus1, synth_minus2, synth_1, synth_2], ignore_index=True)
df_synth['Discrete Thermal Comfort_TA'] = pd.to_numeric(df_synth['Discrete Thermal Comfort_TA'])
print(df_synth['Discrete Thermal Comfort_TA'].value_counts())
df_synth_train_balanced = pd.concat([df_synth, df_feature1_train], ignore_index=True)
print(df_synth_train_balanced['Discrete Thermal Comfort_TA'].value_counts())
print(df_synth_train_balanced)

# approximate to 2 decimal points, as original dataset
df_synth_train_balanced = df_synth_train_balanced.round(2)

print(df_synth_train_balanced)

## Train: train set + synth

In [ ]:
# load best models NB, KNN, SVM, MLP
acc_nb_synth_train_balanced, nb_optimal_synth_train_balanced = buildTrainNB(df_synth_train_balanced, test_size_percentage=test_size_percentage)
acc_knn_synth_train_balanced, knn_optimal_synth_train_balanced = buildTrainKNN(df_synth_train_balanced, test_size_percentage=test_size_percentage)
acc_svm_synth_train_balanced, svm_optimal_synth_train_balanced = buildTrainSVM(df_synth_train_balanced, test_size_percentage=test_size_percentage)

# using the optimal model. re-train in whole train split and test in unseen test split
acc_nb_synth_train_balanced_corr, _ = trainTest_tunedModel(df_feature1_test, nb_optimal_synth_train_balanced)
acc_knn_synth_train_balanced_corr, _ = trainTest_tunedModel(df_feature1_test, knn_optimal_synth_train_balanced)
acc_svm_synth_train_balanced_corr, _ = trainTest_tunedModel(df_feature1_test, svm_optimal_synth_train_balanced)

print("original train")
print(acc_holistic_knn_1)
print(acc_holistic_nb_1)
print(acc_holistic_svm_1)
print("\n")

print("more balanced train")
print(acc_nb_synth_train_balanced_corr)
print(acc_knn_synth_train_balanced_corr)
print(acc_svm_synth_train_balanced_corr)


In [ ]:
"""
original train
0.5885509838998211
0.5974955277280859
0.5974955277280859

# same size synth
more balanced train
0.5974955277280859
0.42397137745974955
0.26654740608228983

# twice synth
0.5974955277280859
0.4525939177101968
0.007155635062611807

# three times synth
0.5974955277280859
0.4203935599284436
0.08050089445438283

"""

## Train: synth

In [ ]:
df_synth = pd.concat([synth_0, synth_minus1, synth_minus2, synth_1, synth_2], ignore_index=True)
df_synth['Discrete Thermal Comfort_TA'] = pd.to_numeric(df_synth['Discrete Thermal Comfort_TA'])
print(df_synth['Discrete Thermal Comfort_TA'].value_counts())
df_synth = df_synth.round(2)

print(df_synth)

# load best models NB, KNN, SVM, MLP
acc_nb_synth, nb_optimal_synth = buildTrainNB(df_synth, test_size_percentage=test_size_percentage)
acc_knn_synth, knn_optimal_synth = buildTrainKNN(df_synth, test_size_percentage=test_size_percentage)
acc_svm_synth, svm_optimal_synth = buildTrainSVM(df_synth, test_size_percentage=test_size_percentage)

# using the optimal model. re-train in whole train split and test in unseen test split
acc_nb_synth, _ = trainTest_tunedModel(df_feature1_test, nb_optimal_synth)
acc_knn_synth, _ = trainTest_tunedModel(df_feature1_test, knn_optimal_synth)
acc_svm_synth, _ = trainTest_tunedModel(df_feature1_test, svm_optimal_synth)

print("original train")
print(acc_holistic_knn_1)
print(acc_holistic_nb_1)
print(acc_holistic_svm_1)
print("\n")

print("more balanced train")
print(acc_nb_synth)
print(acc_knn_synth)
print(acc_svm_synth)

In [ ]:
"""
original train
0.5885509838998211
0.5974955277280859
0.5974955277280859

# same size synth # CHANGE
0.5974955277280859
0.5116279069767442
0.5974955277280859

# twice synth # CHANGE
0.007155635062611807
0.2772808586762075
0.007155635062611807

# three times synth
0.007155635062611807
0.3023255813953488
0.007155635062611807

"""


In [ ]:
all_datasets = pd.DataFrame([
                            # train
                            ['KNN', 'Train', 0.59],
                            ['SVM', 'Train', 0.60],
                            ['NB', 'Train', 0.60],
                            
                            # train + synth1 
                            ['KNN', 'Train+Synth1-TGAN', 0.6],
                            ['SVM', 'Train+Synth1-TGAN', 0.42],
                            ['NB', 'Train+Synth1-TGAN', 0.27],

                            ['KNN', 'Train+Synth1-GANCorr', 0.49],
                            ['SVM', 'Train+Synth1-GANCorr', 0.45],
                            ['NB', 'Train+Synth1-GANCorr', 0.60],

                            ['KNN', 'Train+Synth1-GAN', 0.08],
                            ['SVM', 'Train+Synth1-GAN', 0.49],
                            ['NB', 'Train+Synth1-GAN', 0.60],
    
                            # synth1
                            ['KNN', 'Synth1-TGAN', 0.60],
                            ['SVM', 'Synth1-TGAN', 0.51],
                            ['NB', 'Synth1-TGAN', 0.60],
                            
                            ['KNN', 'Synth1-GANCorr', 0.02],
                            ['SVM', 'Synth1-GANCorr', 0.61],
                            ['NB', 'Synth1-GANCorr', 0.60],
    
                            ['KNN', 'Synth1-GAN', 0.08],
                            ['SVM', 'Synth1-GAN', 0.63],
                            ['NB', 'Synth1-GAN', 0.60],
    
                            # train + synth2
                            ['KNN', 'Train+Synth2-TGAN', 0.6],
                            ['SVM', 'Train+Synth2-TGAN', 0.45],
                            ['NB', 'Train+Synth2-TGAN', 0.01],
                            
                            ['KNN', 'Train+Synth2-GANCorr', 0.49],
                            ['SVM', 'Train+Synth2-GANCorr', 0.42],
                            ['NB', 'Train+Synth2-GANCorr', 0.6],
    
                            ['KNN', 'Train+Synth2-GAN', 0.08],
                            ['SVM', 'Train+Synth2-GAN', 0.42],
                            ['NB', 'Train+Synth2-GAN', 0.08],
    
                            # synth2
                            ['KNN', 'Synth2-TGAN', 0.01],
                            ['SVM', 'Synth2-TGAN', 0.28],
                            ['NB', 'Synth2-TGAN', 0.01],                                                
                            
                            ['KNN', 'Synth2-GANCorr', 0.64],
                            ['SVM', 'Synth2-GANCorr', 0.48],
                            ['NB', 'Synth2-GANCorr', 0.60],  
                            
                            ['KNN', 'Synth2-GAN', 0.08],
                            ['SVM', 'Synth2-GAN', 0.57],
                            ['NB', 'Synth2-GAN', 0.60],  
    
                            # train + synth3
                            ['KNN', 'Train+Synth3-TGAN', 0.60],
                            ['SVM', 'Train+Synth3-TGAN', 0.42],
                            ['NB', 'Train+Synth3-TGAN', 0.08],

                            ['KNN', 'Train+Synth3-GANCorr', 0.28],
                            ['SVM', 'Train+Synth3-GANCorr', 0.37],
                            ['NB', 'Train+Synth3-GANCorr', 0.6],
    
                            ['KNN', 'Train+Synth3-GAN', 0.08],
                            ['SVM', 'Train+Synth3-GAN', 0.46],
                            ['NB', 'Train+Synth3-GAN', 0.08],
    
                            # synth3
                            ['KNN', 'Synth3-TGAN', 0.01],
                            ['SVM', 'Synth3-TGAN', 0.30],
                            ['NB', 'Synth3-TGAN', 0.01],
    
                            ['KNN', 'Synth3-GANCorr', 0.24],
                            ['SVM', 'Synth3-GANCorr', 0.4],
                            ['NB', 'Synth3-GANCorr', 0.27],  
    
                            ['KNN', 'Synth3-GAN', 0.08],
                            ['SVM', 'Synth3-GAN', 0.51],
                            ['NB', 'Synth3-GAN', 0.08] 

                            ], columns = ['Model', 'Training Set and GAN', 'Accuracy'])

# plot heatmap
print("Metric: F1-score micro")
print("Data Split: 20% Test, 70% Train")
all_datasets = all_datasets.pivot("Training Set and GAN", "Model", "Accuracy")
fig = plt.figure(figsize=(12, 10))
sns.set(font_scale=1.5)
ax = sns.heatmap(all_datasets, linewidths = 0.5, annot = True, vmax = 1, cmap = "YlGnBu")
plt.tight_layout()
plt.show()